In [13]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

from implementation import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load and clean the data

In [14]:
y,x,ids=load_csv_data("train.csv") 

In [18]:
# tej colonnes où y'a des -999

x[x==-999]=np.NaN
z=x[:,~np.isnan(x).any(axis=0)] #z is our matrix of observations with size (number of observations, number of features without -999)
#z = build_poly(z,5)
z=one_hot_jet_num(z,17)
print(z)

[[ 51.655  97.827  27.98  ...   1.      0.      0.   ]
 [ 68.768 103.235  48.146 ...   0.      1.      0.   ]
 [162.172 125.953  35.635 ...   0.      1.      0.   ]
 ...
 [ 60.526  75.839  39.757 ...   0.      1.      0.   ]
 [ 19.362  68.812  13.504 ...   0.      0.      1.   ]
 [ 72.756  70.831   7.479 ...   0.      0.      1.   ]]
(250000, 22)


In [19]:
# cross validation for 2 --> TO DO: FOR K

Strain1_z, Stest1_z, Strain1_y, Stest1_y = split_data(z,y,0.8)

# standardize the data (only centered here)

Strain1_z = standardize(Strain1_z)
Stest1_z = standardize(Stest1_z)
Strain1_y = standardize(Strain1_y)
Stest_y = standardize(Stest1_y)

# Calculate MSE for result from least squares 

In [20]:
beta1 = least_squares(Strain1_y, Strain1_z)      #beta from y = X^T*beta --> missing the column of ones'
print(beta1)
res1 = definitive_res(Stest1_z.dot(beta1))
print(res1)
len(Stest1_y[Stest1_y==res1])/len(Stest1_y)

[-2.74674217e-01 -2.76289915e-01 -2.28195706e-02  3.00636886e-01
 -6.90780698e-03 -6.40625008e+02 -2.00102820e-01  1.32371010e-01
  1.24270682e+02 -5.22074739e-04 -1.48474869e-03  1.22541265e+02
 -6.85106131e-04  2.70141344e-03  1.12281935e-01  6.74974947e-05
 -8.37207095e-02  5.42823062e+02 -2.87680585e+02 -4.05676634e+02
 -4.67687208e+02 -4.95392831e+02]
[-1.  1.  1. ... -1.  1. -1.]


0.58546

# Calculate MSE for results from least squares GD and SGD

In [21]:
initial_w = np.zeros(Strain1_z.shape[1])    #w is an array of size = number of features (i.e. number of columns in z)
max_iters = 50
gamma = 0.7

# GD

In [22]:
beta2 = least_squares_GD(Strain1_y, Strain1_z, initial_w, max_iters, gamma)      #beta from y = X*beta --> missing the column of ones'
res2 = definitive_res(Stest1_z.dot(beta2))
len(Stest1_y[Stest1_y==res2])/len(Stest1_y)

0.38028

# SGD

In [23]:
batch_size = 1

beta3 = least_squares_SGD(Strain1_y, Strain1_z, initial_w, batch_size, max_iters, gamma)      #beta from y = X*beta --> missing the column of ones'
res3 = definitive_res(Stest1_z.dot(beta3))
len(Stest1_y[Stest1_y==res3])/len(Stest1_y)

0.57054

# Calculate other LS for Ridge Regression 

In [24]:
lambdas = np.logspace(-5, 0, 15)

# bluid poly
tx_tr = build_poly(Strain1_z, 1)
tx_te = build_poly(Stest1_z, 1)

# ridge regression with different lambda
rmse_tr = []
rmse_te = []
for ind, lambda_ in enumerate(lambdas):
    # ridge regression
    weight = ridge_regression(Strain1_y, tx_tr, lambda_)
    rmse_tr.append(np.sqrt(2 * compute_mse(Strain1_y, tx_tr, weight)))
    rmse_te.append(np.sqrt(2 * compute_mse(Stest1_y, tx_te, weight)))
print(rmse_tr, rmse_te)
print("last weight:",weight)

[0.8797505561792611, 0.8797505591683703, 0.879750574615062, 0.8797506544021134, 0.8797510651817272, 0.8797531640836991, 0.8797637076725343, 0.8798147278826249, 0.8800428738574522, 0.8809179535187134, 0.8835148681631161, 0.8890298065428397, 0.8978370861002744, 0.9106525253549238, 0.9288701699073427] [0.8921614381223276, 0.8921611167268174, 0.8921603946136851, 0.8921587957496518, 0.8921553842177449, 0.8921487738720234, 0.8921394499411098, 0.8921450862129003, 0.8922697693291359, 0.8929136401662627, 0.8950281167170295, 0.8996624634728898, 0.9070934018676545, 0.9180294427302199, 0.9340611968530733]
last weight: [ 7.75014782e-16 -9.25339465e-02 -5.77743345e-03  3.11031369e-02
  2.81534620e-02 -1.30401899e-02  1.82595595e-02 -4.66417316e-02
  6.12505051e-02  5.51222306e-02 -4.54912332e-04 -1.45483676e-03
 -1.24842795e-03 -4.67524618e-05  1.26941254e-03 -4.11684943e-03
  1.23932932e-03  1.00170511e-02  9.23178935e-03 -2.09985537e-02
  3.63262132e-02  1.87208061e-04 -1.77396877e-02]
